In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import random
import re
import json

In [10]:
PAGES = 100

URL = 'https://old.reddit.com/r/datascience/top/?sort=top&t=year'

USER_AGENT_LIST = [

    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',

    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [14]:
def html_data(url, user_agent_list):
    
    users = {}
    
    users['title'] = []
    users['authors'] = []
    users['comments'] = []
    users['likes'] = []
    users['dislikes'] = []
    users['dates'] = []
    
    for i in range(1, PAGES):
        
        user_agent = random.choice(user_agent_list)
        
        headers = {'User-Agent': user_agent}
        
        html = requests.get(url, headers = headers)
        
        soup = BeautifulSoup(html.text, 'html.parser')
        
        threads = soup.find_all('span', class_='domain')
        
        #for thread in threads:
        #    if thread not in ['(self.datascience)', 'i.redd.it', 'v.redd.it']:
        #        continue
        #    parent_div = thread.parent.parent.parent.parent
        
        status_code = html.status_code
        
        domains, titles, authors, comments, likes, dislikes, dates = ([] for i in range(7))
        
        attrs = {'class': 'thing', 'data-domain': ['self.datascience', 'i.redd.it', 'v.redd.it']}
        
        for post in soup.find_all('div', attrs=attrs):
            
            try:
                domains.append(post.attrs['data-domain'])
            except:
                domains.append('Unknown')
            try:
                titles.append(post.find('p', class_='title').text)
            except:
                titles.append('Unknown')
            try:
                authors.append(post.find('a', class_='author').text)
            except:
                authors.append('Unknown')
            try:
                comments.append(post.find('a', class_='comments').text.split()[0])
            except:
                comments.append(0)
            try:
                likes.append(post.find('div', attrs={'class': 'score likes'}).text)
            except:
                likes.append(0)
            try:
                dislikes.append(post.find('div', attrs={'class': 'score dislikes'}).text)
            except:
                likes.append(0)
                
            date = post.find('p', attrs={'class': 'tagline'})    
            date = date.time.prettify()
            dates.append(re.findall(r'\d{4}\-\d{2}\-\w+\:\d+\:\d+\+\d+' , date)[0])

            
        users['title'] += titles
        users['authors'] += authors
        users['comments'] += comments
        users['likes'] += likes
        users['dislikes'] += dislikes
        users['dates'] += dates
        
        next_button = soup.find('span', class_='next-button')
        
        try:
            url = next_button.find('a').attrs['href']
        except:
            break
            
        time.sleep(2)

    
    return users

In [15]:
saved = html_data(URL, USER_AGENT_LIST)

In [86]:
with open('result.json', 'w') as fp:
    json.dump(saved, fp)
    fp.close()

In [16]:
len(saved['title'])

845

In [17]:
saved['title']

['It’s never too early (i.redd.it)',
 'Imposter Syndrome is a problem for me and I think this is the main contributor (i.redd.it)',
 'Fun/TriviaTrue that (i.redd.it)',
 'Fun/TriviaDS at a glance (i.redd.it)',
 'Fun/TriviaGraph of graph analysis (i.redd.it)',
 'When the boss doesn’t like your charts (i.redd.it)',
 "ToolingFor any python & pandas users out there, here's a free tool to visualize your dataframes (v.redd.it)",
 'Fun/TriviaThe next time my coworkers ask what metrics I used for my model. (v.redd.it)',
 'Fun/TriviaFit an exponential curve to anything... (i.redd.it)',
 "Beware of today's data (i.redd.it)",
 "Fun/TriviaIt's Meme Monday, so here's a python meme for DS folks (i.redd.it)",
 'Fun/TriviaExpectation vs reality (i.redd.it)',
 'Found this (i.redd.it)',
 'Fun/TriviaWhen you get an Excel Sheet of 1000x5 and your clients ask you to do "Data Science" on this with "AI" (i.redd.it)',
 'Fun/TriviaRecruiters be like (i.redd.it)',
 'The formatting struggle. (i.redd.it)',
 'Fun/T